## Context
### Business value for customers 
Gen AI driven solution leads to almost 50% cost savings.
### TAM in India 
### Current Adoption
#### Global Success Stories
#### Success Stories in India
#### Recommended Architecture
#### BDM decks
#### TDM decks
#### Accelerators
#### Blogs
#### Existing Demos
#### New Demo
#### Open Issues Currently In Progress


https://openai.com/index/hello-gpt-4o/
https://techcommunity.microsoft.com/t5/ai-azure-ai-services-blog/fast-transcription-public-preview-in-azure-ai-speech/ba-p/4108720

# Voice BOT 
## The old way:
1. Sales Agent Persona - generates response using Open AI 
2. Response is converted to Speech 
3. Customer Reverts in audio 
4. Speech is converted to text - Average time for 1 + 2 + 4 is 5.4 s with GPT4 and 2.8 with GPT-3.5
5. Back to step 1
## The new way:
1. Sales Agent Persona - generates audio response using Open AI - average expected time 320ms
2. Customer Reverts in audio
3. Back to step 1


Latency Reduction 
1. Streaming 
2. Fast Transcription 
3. Compression of audio - increased speed of words spoken
4. Containers 

In [1]:
import openai
import azure.cognitiveservices.speech as speechsdk
from azure.cognitiveservices.speech import audio
import simpleaudio as sa
import io
from pydub import AudioSegment
import gradio as gr
from io import BytesIO
import tempfile

import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import wave

from speech_demo_set_default_prompts import set_default_prompts_loan
from speech_demo_set_default_prompts import set_default_prompts_ecom 
from speech_demo_gen_avatars import gen_dalle_img
from speech_demo_get_conv_summary import get_summary
from speech_demo_openai_response import get_openai_resp_text_audio, final_segment
from speech_demo_human_response import get_human_resp_text_audio
import io
import azure.cognitiveservices.speech as speechsdk
from pydub import AudioSegment, silence
import simpleaudio as sa
import threading
import asyncio

load_dotenv(override=True)

speech_ep = os.getenv('SPEECH_EP')
speech_key = os.getenv('SPEECH_KEY')
speech_region = os.getenv('SPEECH_REGION')

summary_prompt = ""
person1_prompt = ""
person2_prompt = ""

In [2]:
#gen_dalle_img("ecom","persona2")

In [3]:
def gen_dalle_img_all(theme):
    img1 = gen_dalle_img(theme,"persona1")
    img2 = gen_dalle_img(theme,"persona2")
    return img1, img2

In [4]:
async def main(language, agent_msgs, customer_msgs, response_mode):
    #global person1_prompt
    #global person2_prompt
    #global agent_msgs
    #global customer_msgs
    i = 0
    #agent_msgs = [{"role":"system","content":person1_prompt}]
    #customer_msgs = [{"role":"system","content":person2_prompt}]
    while i < 10:
        print("Calling agent conversation")
        #print(agent_msgs)
        await get_openai_resp_text_audio(agent_msgs, customer_msgs, language, True)
        if "Thank" in agent_msgs[-1]["content"]:
            print("Conversation over!")
            break
        if "thank" in agent_msgs[-1]["content"]:
            print("Conversation over!")
            break
        if "धन्यवाद" in agent_msgs[-1]["content"]:
            print("Conversation over!")
            break
        print("Calling customer conversation")
        
        if response_mode == "Customer Speaks":
            # Process the audio input from the user
            wav_file = f"customer_response_{i}.wav"
            if language == "English":
                lang = "en-IN"
            else:
                lang = "hi-IN"
            customer_response = get_human_resp_text_audio(lang, wav_file)
            agent_msgs.append({"role": "user", "content": customer_response})
        else:
            await get_openai_resp_text_audio(agent_msgs, customer_msgs, language, False)
            
        i += 1



In [5]:
def run_script(language, additional_prompt1, additional_prompt2, response_mode):
    #global agent_msgs, customer_msgs
    global final_segment
    #global summary_prompt
    
    #person1_prompt, person2_prompt = set_prompt(language, theme)
    agent_msgs = [{"role":"system", "content": additional_prompt1}]
    customer_msgs = [{"role":"system", "content": additional_prompt2}]

    
    try:
        import nest_asyncio
        nest_asyncio.apply()
    except ImportError:
        pass

    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(main(language, agent_msgs, customer_msgs, response_mode))
    
    final_segment.export("persona.wav", format="wav")
    summary = get_summary(language, agent_msgs)
    return summary, "persona.wav"


def run_main_loop(language, additional_prompt1, additional_prompt2, response_mode):
    return run_script(language, additional_prompt1, additional_prompt2, response_mode)


In [6]:
def set_prompt(language, theme):
    #language = "Hindi"
    if theme == "ecom":
        person1_prompt, person2_prompt = set_default_prompts_ecom(language)
    else:
        person1_prompt, person2_prompt = set_default_prompts_loan(language)
    img1, img2 = gen_dalle_img_all(theme)
    return person1_prompt, person2_prompt, img1, img2 


with gr.Blocks() as iface:
    with gr.Row():
        language = gr.Dropdown(choices=["English", "Hindi"], label="Select Language")
        theme = gr.Dropdown(choices=["ecom", "loan"], label="Select Theme")
        lang_button = gr.Button("Set Prompts")
    with gr.Row():
        additional_prompt1 = gr.Textbox(lines=5, placeholder="Additional Prompt 1", label="Additional Prompt 1")
        additional_prompt2 = gr.Textbox(lines=5, placeholder="Additional Prompt 2", label="Additional Prompt 2")
    with gr.Row():
        response_mode = gr.Dropdown(choices=["Customer Speaks", "OpenAI Generates"], label="Response Mode")
        run_button = gr.Button("Run")
    with gr.Row():
            with gr.Column():
                person1_image = gr.Image(label="Person 1 Image")
            with gr.Column():
                summary = gr.Textbox(lines=10, placeholder="Conversation Summary", label="Conversation Summary")
            with gr.Column():
                person2_image = gr.Image(label="Person 2 Image")
                

    with gr.Row():     
            person1_audio = gr.Audio(label="Person 1 Audio")
            person2_audio = gr.Audio(label="Person 2 Audio")
                
    
    
       
    lang_button.click(fn=set_prompt, inputs=[language, theme], outputs=[additional_prompt1, additional_prompt2, person1_image, person2_image])
    run_button.click(fn=run_main_loop, inputs=[language, additional_prompt1, additional_prompt2, response_mode], outputs=[summary, person1_audio])
    
iface.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


Calling agent conversation
Received text chunk: Hello
Received text chunk: !
Received text chunk:  Are
Received text chunk:  you
Received text chunk:  interested
Received text chunk:  in
Received text chunk:  hearing
Received text chunk:  about
Received text chunk:  a
Received text chunk:  new
Received text chunk:  offer
Received text chunk:  we
Received text chunk:  have
Received text chunk: ?
Speech synthesized for text [Hello!]
Speech synthesized for text [Are you interested in hearing about a new offer we have?]
Calling customer conversation
no callback
Monitoring microphone... Start speaking to begin transcription.
Speech detected, starting transcription...
SESSION STARTED: SessionEventArgs(session_id=be3f767f4da44c0185237a7ec07295d8)
Recording... Press Ctrl+C to stop.
True no
True no i
True no i am kind of
True no i am kind of busy
True no i am kind of busy right now
Recognized: False No, I am kind of busy right now.
SESSION STOPPED: SessionEventArgs(session_id=be3f767f4da44c0185